In [1]:
from vnpy.app.cta_strategy.backtesting import BacktestingEngine, OptimizationSetting
from vnpy.app.cta_strategy.base import BacktestingMode
from datetime import datetime
from vnpy.trader.constant import Interval, Exchange
# from vnpy.app.atr_rsi_strategy import AtrRsiStrategy

from vnpy.app.portfolio_strategy.backtesting import BacktestingEngine as PairBacktestEngine

import matplotlib
%matplotlib inline

# 講座：交易成本實作｜手續費、交易稅和滑價成本的計算

In [2]:
import re


def getCommission(symbol,
                  exchange,
                  cost,
                  multiplier,
                  qty,
                  Real=False,
                  direction=''):
    """計算個別部位的單邊交易成本"""
    try:
        if exchange == 'TFE':
            if re.match('MTX[0-9]*', symbol):  # 小台指期
                return 50 + (1 * 50 + cost * (2 / 100000) * multiplier
                             ) * qty if not Real else 50 + (cost *
                                                            (2 / 100000) *
                                                            multiplier) * qty
            elif re.match('TX[0-9]*',
                          symbol) or re.match('TE[0-9]*', symbol) or re.match(
                              'TF[0-9]*', symbol):  # 大台指期
                return 50 + (1 * 200 + cost * (2 / 100000) * multiplier
                             ) * qty if not Real else 50 + (cost *
                                                            (2 / 100000) *
                                                            multiplier) * qty
            elif re.match(r"TX[O0-9][0-9]*[A-Z][0-9]", symbol):  # 台指選
                return 30 + (getMinimumTickOpt(cost) + cost * 2 /
                             100000) * qty * multiplier if not Real else 30 + (
                                 cost * 2 / 100000) * qty * multiplier
            elif re.match(r"[A-Z][A-Z]F[0-9][0-9]", symbol):  # 個股期貨
                return 50 + (
                    getMinimumTick(cost) + cost *
                    (2 / 100000)) * qty * multiplier if not Real else 50 + (
                        cost * (2 / 100000)) * qty * multiplier
        elif exchange == 'TSE':
            tick = getMinimumTick(cost)
            commission = cost * (0.1425 / 100) * multiplier
            commission = 20 if commission < 20 else commission
            fee = cost * (0.3 / 100) * multiplier
            slide = tick * multiplier
            if direction == 'EXIT' or direction == 0:
                return (commission + fee +
                        slide) * qty if not Real else (commission + fee) * qty
            else:
                return (commission +
                        slide) * qty if not Real else commission * qty
    except Exception as e:
        print(e)


def getMinimumTickOpt(cost):
    if cost < 10:
        return 0.1
    elif cost < 50:
        return 0.5
    elif cost < 500:
        return 1
    elif cost < 1000:
        return 5
    else:
        return 10


def getMinimumTick(cost):
    if cost < 10:
        return 0.01
    elif cost < 50:
        return 0.05
    elif cost < 100:
        return 0.1
    elif cost < 500:
        return 0.5
    elif cost < 1000:
        return 1
    else:
        return 5

# 講座：開發第一個策略

In [10]:
from vnpy.app.cta_strategy import (
    CtaTemplate,
    StopOrder,
    TickData,
    BarData,
    TradeData,
    OrderData,
    BarGenerator,
    ArrayManager,
)


class DoubleMaStrategy(CtaTemplate):
    author = "用Python的交易员"

    fast_window = 165#10
    slow_window = 5#20

    fast_ma0 = 0.0
    fast_ma1 = 0.0

    slow_ma0 = 0.0
    slow_ma1 = 0.0

    parameters = ["fast_window", "slow_window"]
    variables = ["fast_ma0", "fast_ma1", "slow_ma0", "slow_ma1"]

    def __init__(self, cta_engine, strategy_name, vt_symbol, setting):
        """"""
        super().__init__(cta_engine, strategy_name, vt_symbol, setting)

        self.bg = BarGenerator(self.on_bar)
        self.am = ArrayManager(max(self.fast_window, self.slow_window)+1)

    def on_init(self):
        """
        Callback when strategy is inited.
        """
        self.write_log("策略初始化")
        self.load_bar(10)

    def on_start(self):
        """
        Callback when strategy is started.
        """
        self.write_log("策略启动")
        self.put_event()

    def on_stop(self):
        """
        Callback when strategy is stopped.
        """
        self.write_log("策略停止")

        self.put_event()

    def on_tick(self, tick: TickData):
        """
        Callback of new tick data update.
        """
        self.bg.update_tick(tick)

    def on_bar(self, bar: BarData):
        """
        Callback of new bar data update.
        """

        am = self.am
        am.update_bar(bar)
        if not am.inited:
            return

        fast_ma = am.sma(self.fast_window, array=True)
        self.fast_ma0 = fast_ma[-1]
        self.fast_ma1 = fast_ma[-2]

        slow_ma = am.sma(self.slow_window, array=True)
        self.slow_ma0 = slow_ma[-1]
        self.slow_ma1 = slow_ma[-2]

        cross_over = self.fast_ma0 > self.slow_ma0 and self.fast_ma1 < self.slow_ma1
        cross_below = self.fast_ma0 < self.slow_ma0 and self.fast_ma1 > self.slow_ma1

        if cross_over:
            if self.pos == 0:
                self.buy(bar.close_price, 1)
#             elif self.pos < 0:
#                 self.cover(bar.close_price, 1)
#                 self.buy(bar.close_price, 1)

        elif cross_below:
#             if self.pos == 0:
#                 self.short(bar.close_price, 1)
            if self.pos > 0:
                self.sell(bar.close_price, 1)
#                 self.short(bar.close_price, 1)

        self.put_event()

    def on_order(self, order: OrderData):
        """
        Callback of new order data update.
        """
        pass

    def on_trade(self, trade: TradeData):
        """
        Callback of new trade data update.
        """
        self.put_event()

    def on_stop_order(self, stop_order: StopOrder):
        """
        Callback of stop order update.
        """
        pass


# 講座：歷史數據回測與績效驗證


In [18]:
# from vnpy.trader.constant import Interval, Exchange
from functools import partial



tickers = '2330.TSE'
symbol, exchange = tickers.split('.')
get_commission = partial(getCommission, symbol=symbol, exchange=exchange)
engine = BacktestingEngine()

engine.set_parameters(
    vt_symbol=tickers,
    interval=Interval.DAILY,
    start=datetime(2015, 1, 1),
    end=datetime(2021, 1, 19),
    rate=0,#get_commission, # 手續費 
    slippage=0,#getMinimumTick,  # 滑價
    size=1e3, # 股票價值 1000
    pricetick=getMinimumTick, # 價格變動單位
    capital=5e5, # 回測本金 1_000_000
    mode=BacktestingMode.BAR,
    collection_name='TSE')

setting = {
    'fast_window' : 8,
    'slow_window' : 54,
}

engine.add_strategy(DoubleMaStrategy, setting)
engine.load_data()
engine.run_backtesting()
df = engine.calculate_result()
engine.calculate_statistics()
engine.show_chart()

2021-01-29 16:09:43.663204	开始加载历史数据
2021-01-29 16:09:43.664204	加载进度： [1%]
2021-01-29 16:09:43.664204	加载进度： [3%]
2021-01-29 16:09:43.664204	加载进度： [4%]
2021-01-29 16:09:43.664204	加载进度： [5%]
2021-01-29 16:09:43.664204	加载进度： [7%]
2021-01-29 16:09:43.664204	加载进度： [8%]
2021-01-29 16:09:43.664204	加载进度： [10%]
2021-01-29 16:09:43.664204	加载进度：# [11%]
2021-01-29 16:09:43.664204	加载进度：# [12%]
2021-01-29 16:09:43.664204	加载进度：# [14%]
2021-01-29 16:09:43.664204	加载进度：# [15%]
2021-01-29 16:09:43.664204	加载进度：# [16%]
2021-01-29 16:09:43.664204	加载进度：# [18%]
2021-01-29 16:09:43.664204	加载进度：# [19%]
2021-01-29 16:09:43.664204	加载进度：## [20%]
2021-01-29 16:09:43.665204	加载进度：## [22%]
2021-01-29 16:09:43.665204	加载进度：## [23%]
2021-01-29 16:09:43.665204	加载进度：## [24%]
2021-01-29 16:09:43.665204	加载进度：## [26%]
2021-01-29 16:09:43.665204	加载进度：## [27%]
2021-01-29 16:09:43.665204	加载进度：## [29%]
2021-01-29 16:09:43.665204	加载进度：## [30%]
2021-01-29 16:09:43.665204	加载进度：### [31%]
2021-01-29 16:09:43.665204	加载进度：### [33%]
2021-

# 講座：策略參數優化


In [13]:
setting = OptimizationSetting()
setting.set_target("total_net_pnl") # max_drawdown, sharpe_ratio
setting.add_parameter("fast_window", 5, 30, 1)
setting.add_parameter("slow_window", 20, 120, 1)

result = engine.run_ga_optimization(setting)

2021-01-29 16:05:28.803627	参数优化空间：2626
2021-01-29 16:05:28.804627	每代族群总数：100
2021-01-29 16:05:28.804627	优良筛选个数：80
2021-01-29 16:05:28.804627	迭代次数：30
2021-01-29 16:05:28.804627	交叉概率：95%
2021-01-29 16:05:28.804627	突变概率：5%
2021-01-29 16:05:28.804627	开始加载历史数据
2021-01-29 16:05:28.804627	加载进度： [1%]
2021-01-29 16:05:28.804627	加载进度： [1%]
2021-01-29 16:05:28.804627	加载进度： [2%]
2021-01-29 16:05:28.804627	加载进度： [3%]
2021-01-29 16:05:28.805627	加载进度： [4%]
2021-01-29 16:05:28.805627	加载进度： [4%]
2021-01-29 16:05:28.805627	加载进度： [5%]
2021-01-29 16:05:28.805627	加载进度： [6%]
2021-01-29 16:05:28.805627	加载进度： [7%]
2021-01-29 16:05:28.805627	加载进度： [7%]
2021-01-29 16:05:28.805627	加载进度： [8%]
2021-01-29 16:05:28.805627	加载进度： [9%]
2021-01-29 16:05:28.805627	加载进度： [10%]
2021-01-29 16:05:28.805627	加载进度：# [10%]
2021-01-29 16:05:28.805627	加载进度：# [11%]
2021-01-29 16:05:28.805627	加载进度：# [12%]
2021-01-29 16:05:28.805627	加载进度：# [13%]
2021-01-29 16:05:28.805627	加载进度：# [13%]
2021-01-29 16:05:28.805627	加载进度：# [14%]
2021-01-2

2021-01-29 16:05:29.723680	历史数据回放结束
2021-01-29 16:05:29.723680	开始计算逐日盯市盈亏
2021-01-29 16:05:29.758682	逐日盯市盈亏计算完成
2021-01-29 16:05:29.759682	开始计算策略统计指标
2021-01-29 16:05:29.798684	策略统计指标计算完成
2021-01-29 16:05:29.799684	开始加载历史数据
2021-01-29 16:05:29.799684	加载进度： [1%]
2021-01-29 16:05:29.799684	加载进度： [1%]
2021-01-29 16:05:29.800684	加载进度： [2%]
2021-01-29 16:05:29.800684	加载进度： [3%]
2021-01-29 16:05:29.800684	加载进度： [4%]
2021-01-29 16:05:29.800684	加载进度： [4%]
2021-01-29 16:05:29.801684	加载进度： [5%]
2021-01-29 16:05:29.801684	加载进度： [6%]
2021-01-29 16:05:29.801684	加载进度： [7%]
2021-01-29 16:05:29.802684	加载进度： [7%]
2021-01-29 16:05:29.802684	加载进度： [8%]
2021-01-29 16:05:29.802684	加载进度： [9%]
2021-01-29 16:05:29.802684	加载进度： [10%]
2021-01-29 16:05:29.803684	加载进度：# [10%]
2021-01-29 16:05:29.803684	加载进度：# [11%]
2021-01-29 16:05:29.803684	加载进度：# [12%]
2021-01-29 16:05:29.803684	加载进度：# [13%]
2021-01-29 16:05:29.803684	加载进度：# [13%]
2021-01-29 16:05:29.803684	加载进度：# [14%]
2021-01-29 16:05:29.803684	加载进度：# [15%]
2

E:\vnpy_env\vnpy\app\cta_strategy\backtesting.py:399: RuntimeWarning:

invalid value encountered in log



2021-01-29 16:05:29.961693	历史数据回放结束
2021-01-29 16:05:29.963693	开始计算逐日盯市盈亏
2021-01-29 16:05:29.998695	逐日盯市盈亏计算完成
2021-01-29 16:05:29.999695	开始计算策略统计指标
2021-01-29 16:05:30.028697	策略统计指标计算完成
2021-01-29 16:05:30.029697	开始加载历史数据
2021-01-29 16:05:30.029697	加载进度： [1%]
2021-01-29 16:05:30.030697	加载进度： [1%]
2021-01-29 16:05:30.030697	加载进度： [2%]
2021-01-29 16:05:30.030697	加载进度： [3%]
2021-01-29 16:05:30.031697	加载进度： [4%]
2021-01-29 16:05:30.031697	加载进度： [4%]
2021-01-29 16:05:30.031697	加载进度： [5%]
2021-01-29 16:05:30.031697	加载进度： [6%]
2021-01-29 16:05:30.032697	加载进度： [7%]
2021-01-29 16:05:30.032697	加载进度： [7%]
2021-01-29 16:05:30.032697	加载进度： [8%]
2021-01-29 16:05:30.032697	加载进度： [9%]
2021-01-29 16:05:30.033697	加载进度： [10%]
2021-01-29 16:05:30.033697	加载进度：# [10%]
2021-01-29 16:05:30.033697	加载进度：# [11%]
2021-01-29 16:05:30.033697	加载进度：# [12%]
2021-01-29 16:05:30.033697	加载进度：# [13%]
2021-01-29 16:05:30.033697	加载进度：# [13%]
2021-01-29 16:05:30.033697	加载进度：# [14%]
2021-01-29 16:05:30.034697	加载进度：# [15%]
2

2021-01-29 16:05:30.736738	历史数据回放结束
2021-01-29 16:05:30.739738	开始计算逐日盯市盈亏
2021-01-29 16:05:30.786740	逐日盯市盈亏计算完成
2021-01-29 16:05:30.788740	开始计算策略统计指标
2021-01-29 16:05:30.829743	策略统计指标计算完成
2021-01-29 16:05:30.831743	开始加载历史数据
2021-01-29 16:05:30.832743	加载进度： [1%]
2021-01-29 16:05:30.832743	加载进度： [1%]
2021-01-29 16:05:30.833743	加载进度： [2%]
2021-01-29 16:05:30.833743	加载进度： [3%]
2021-01-29 16:05:30.834743	加载进度： [4%]
2021-01-29 16:05:30.834743	加载进度： [4%]
2021-01-29 16:05:30.835743	加载进度： [5%]
2021-01-29 16:05:30.835743	加载进度： [6%]
2021-01-29 16:05:30.836743	加载进度： [7%]
2021-01-29 16:05:30.836743	加载进度： [7%]
2021-01-29 16:05:30.836743	加载进度： [8%]
2021-01-29 16:05:30.837743	加载进度： [9%]
2021-01-29 16:05:30.837743	加载进度： [10%]
2021-01-29 16:05:30.837743	加载进度：# [10%]
2021-01-29 16:05:30.838743	加载进度：# [11%]
2021-01-29 16:05:30.838743	加载进度：# [12%]
2021-01-29 16:05:30.838743	加载进度：# [13%]
2021-01-29 16:05:30.839743	加载进度：# [13%]
2021-01-29 16:05:30.839743	加载进度：# [14%]
2021-01-29 16:05:30.840743	加载进度：# [15%]
2

...

2021-01-29 16:07:08.817347	策略初始化完成
2021-01-29 16:07:08.817347	开始回放历史数据
2021-01-29 16:07:09.021359	历史数据回放结束
2021-01-29 16:07:09.023359	开始计算逐日盯市盈亏
2021-01-29 16:07:09.056361	逐日盯市盈亏计算完成
2021-01-29 16:07:09.058361	开始计算策略统计指标
2021-01-29 16:07:09.076362	策略统计指标计算完成
2021-01-29 16:07:09.076362	开始加载历史数据
2021-01-29 16:07:09.076362	加载进度： [1%]
2021-01-29 16:07:09.076362	加载进度： [1%]
2021-01-29 16:07:09.076362	加载进度： [2%]
2021-01-29 16:07:09.076362	加载进度： [3%]
2021-01-29 16:07:09.076362	加载进度： [4%]
2021-01-29 16:07:09.076362	加载进度： [4%]
2021-01-29 16:07:09.076362	加载进度： [5%]
2021-01-29 16:07:09.077362	加载进度： [6%]
2021-01-29 16:07:09.077362	加载进度： [7%]
2021-01-29 16:07:09.077362	加载进度： [7%]
2021-01-29 16:07:09.077362	加载进度： [8%]
2021-01-29 16:07:09.077362	加载进度： [9%]
2021-01-29 16:07:09.077362	加载进度： [10%]
2021-01-29 16:07:09.077362	加载进度：# [10%]
2021-01-29 16:07:09.077362	加载进度：# [11%]
2021-01-29 16:07:09.077362	加载进度：# [12%]
2021-01-29 16:07:09.078362	加载进度：# [13%]
2021-01-29 16:07:09.078362	加载进度：# [13%]
2021-01-29

2021-01-29 16:07:09.474385	历史数据回放结束
2021-01-29 16:07:09.475385	开始计算逐日盯市盈亏
2021-01-29 16:07:09.508387	逐日盯市盈亏计算完成
2021-01-29 16:07:09.509387	开始计算策略统计指标
2021-01-29 16:07:09.525388	策略统计指标计算完成
28 	100   	[868800.]        	[0.]             	[868800.]     	[868800.]
2021-01-29 16:07:09.541389	开始加载历史数据
2021-01-29 16:07:09.542389	加载进度： [1%]
2021-01-29 16:07:09.542389	加载进度： [1%]
2021-01-29 16:07:09.542389	加载进度： [2%]
2021-01-29 16:07:09.542389	加载进度： [3%]
2021-01-29 16:07:09.542389	加载进度： [4%]
2021-01-29 16:07:09.542389	加载进度： [4%]
2021-01-29 16:07:09.542389	加载进度： [5%]
2021-01-29 16:07:09.543389	加载进度： [6%]
2021-01-29 16:07:09.543389	加载进度： [7%]
2021-01-29 16:07:09.543389	加载进度： [7%]
2021-01-29 16:07:09.543389	加载进度： [8%]
2021-01-29 16:07:09.543389	加载进度： [9%]
2021-01-29 16:07:09.543389	加载进度： [10%]
2021-01-29 16:07:09.543389	加载进度：# [10%]
2021-01-29 16:07:09.543389	加载进度：# [11%]
2021-01-29 16:07:09.543389	加载进度：# [12%]
2021-01-29 16:07:09.543389	加载进度：# [13%]
2021-01-29 16:07:09.543389	加载进度：# [13%]
2021-01-2

2021-01-29 16:07:09.947412	历史数据回放结束
2021-01-29 16:07:09.947412	开始计算逐日盯市盈亏
2021-01-29 16:07:09.984414	逐日盯市盈亏计算完成
2021-01-29 16:07:09.985414	开始计算策略统计指标
2021-01-29 16:07:10.003415	策略统计指标计算完成
29 	100   	[868800.]        	[0.]             	[868800.]     	[868800.]
2021-01-29 16:07:10.019416	开始加载历史数据
2021-01-29 16:07:10.019416	加载进度： [1%]
2021-01-29 16:07:10.020416	加载进度： [1%]
2021-01-29 16:07:10.020416	加载进度： [2%]
2021-01-29 16:07:10.020416	加载进度： [3%]
2021-01-29 16:07:10.020416	加载进度： [4%]
2021-01-29 16:07:10.020416	加载进度： [4%]
2021-01-29 16:07:10.020416	加载进度： [5%]
2021-01-29 16:07:10.020416	加载进度： [6%]
2021-01-29 16:07:10.021416	加载进度： [7%]
2021-01-29 16:07:10.021416	加载进度： [7%]
2021-01-29 16:07:10.021416	加载进度： [8%]
2021-01-29 16:07:10.021416	加载进度： [9%]
2021-01-29 16:07:10.021416	加载进度： [10%]
2021-01-29 16:07:10.021416	加载进度：# [10%]
2021-01-29 16:07:10.021416	加载进度：# [11%]
2021-01-29 16:07:10.021416	加载进度：# [12%]
2021-01-29 16:07:10.021416	加载进度：# [13%]
2021-01-29 16:07:10.021416	加载进度：# [13%]
2021-01-2

2021-01-29 16:07:10.396438	历史数据回放结束
2021-01-29 16:07:10.396438	开始计算逐日盯市盈亏
2021-01-29 16:07:10.427440	逐日盯市盈亏计算完成
2021-01-29 16:07:10.428440	开始计算策略统计指标
2021-01-29 16:07:10.452441	策略统计指标计算完成
2021-01-29 16:07:10.453441	开始加载历史数据
2021-01-29 16:07:10.453441	加载进度： [1%]
2021-01-29 16:07:10.453441	加载进度： [1%]
2021-01-29 16:07:10.454441	加载进度： [2%]
2021-01-29 16:07:10.454441	加载进度： [3%]
2021-01-29 16:07:10.454441	加载进度： [4%]
2021-01-29 16:07:10.455441	加载进度： [4%]
2021-01-29 16:07:10.455441	加载进度： [5%]
2021-01-29 16:07:10.455441	加载进度： [6%]
2021-01-29 16:07:10.456441	加载进度： [7%]
2021-01-29 16:07:10.456441	加载进度： [7%]
2021-01-29 16:07:10.456441	加载进度： [8%]
2021-01-29 16:07:10.456441	加载进度： [9%]
2021-01-29 16:07:10.457441	加载进度： [10%]
2021-01-29 16:07:10.457441	加载进度：# [10%]
2021-01-29 16:07:10.457441	加载进度：# [11%]
2021-01-29 16:07:10.457441	加载进度：# [12%]
2021-01-29 16:07:10.458441	加载进度：# [13%]
2021-01-29 16:07:10.458441	加载进度：# [13%]
2021-01-29 16:07:10.458441	加载进度：# [14%]
2021-01-29 16:07:10.458441	加载进度：# [15%]
2

2021-01-29 16:07:11.083477	历史数据回放结束
2021-01-29 16:07:11.084477	开始计算逐日盯市盈亏
2021-01-29 16:07:11.116479	逐日盯市盈亏计算完成
2021-01-29 16:07:11.118479	开始计算策略统计指标
2021-01-29 16:07:11.133480	策略统计指标计算完成
2021-01-29 16:07:11.134480	开始加载历史数据
2021-01-29 16:07:11.134480	加载进度： [1%]
2021-01-29 16:07:11.135480	加载进度： [1%]
2021-01-29 16:07:11.135480	加载进度： [2%]
2021-01-29 16:07:11.135480	加载进度： [3%]
2021-01-29 16:07:11.136480	加载进度： [4%]
2021-01-29 16:07:11.136480	加载进度： [4%]
2021-01-29 16:07:11.136480	加载进度： [5%]
2021-01-29 16:07:11.137480	加载进度： [6%]
2021-01-29 16:07:11.137480	加载进度： [7%]
2021-01-29 16:07:11.137480	加载进度： [7%]
2021-01-29 16:07:11.137480	加载进度： [8%]
2021-01-29 16:07:11.138480	加载进度： [9%]
2021-01-29 16:07:11.138480	加载进度： [10%]
2021-01-29 16:07:11.138480	加载进度：# [10%]
2021-01-29 16:07:11.138480	加载进度：# [11%]
2021-01-29 16:07:11.139480	加载进度：# [12%]
2021-01-29 16:07:11.139480	加载进度：# [13%]
2021-01-29 16:07:11.139480	加载进度：# [13%]
2021-01-29 16:07:11.140480	加载进度：# [14%]
2021-01-29 16:07:11.140480	加载进度：# [15%]
2

In [14]:
result

[({'fast_window': 8, 'slow_window': 54}, 868800.0, {})]